In [31]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [8]:
# Url for data
link = """https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"""
url = f"""{link}?raw=1"""

In [9]:
df = pd.read_csv(url)

In [10]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
1825,Some suggested titles for listing the Machine ...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,What are some suggested titles for listing the...,machine-learning-zoomcamp
1826,It is best advised that you do not list the Ma...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,Should I list the Machine Learning Zoomcamp ex...,machine-learning-zoomcamp
1827,You can incorporate your Machine Learning Zoom...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,In which LinkedIn sections can I incorporate m...,machine-learning-zoomcamp
1828,The advice on including a project link in a CV...,I’ve seen LinkedIn users list DataTalksClub as...,c6a22665,Who gave advice on including a project link in...,machine-learning-zoomcamp


In [12]:
df = df.iloc[:300]

In [13]:
from sentence_transformers import SentenceTransformer
model_name = "multi-qa-mpnet-base-dot-v1"
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
answer_llm = df.iloc[0].answer_llm

In [17]:
answer_emb = embedding_model.encode(answer_llm)

In [19]:
# First answer
answer_emb[0]

-0.42244655

In [28]:
# Evaluating the result for dot product
evaluations = []

for i in tqdm(range(df.shape[0])):
    orig_ans = embedding_model.encode(df['answer_orig'][i])
    llm_ans = embedding_model.encode(df['answer_llm'][i])
    score = orig_ans.dot(llm_ans)
    evaluations.append(score)

100%|██████████| 300/300 [02:23<00:00,  2.08it/s]


In [29]:
def calculate_percentile(scores, percentile):
    scores.sort()
    N = len(scores)
    R = percentile * (N + 1)
    
    if R.is_integer():
        return scores[int(R) - 1]
    else:
        k = int(R)
        d = R - k
        return scores[k - 1] + d * (scores[k] - scores[k - 1])

In [30]:
# Dot product's percentile
percentile_75 = calculate_percentile(evaluations, 0.75)
print("75th Percentile:", percentile_75)

75th Percentile: 31.727840423583984


In [32]:
# vector normalization 
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [33]:
# Evaluating the result for cosine
cosine_evaluations = []

for i in tqdm(range(df.shape[0])):
    orig_ans = normalize_vector(embedding_model.encode(df['answer_orig'][i]))
    llm_ans = normalize_vector(embedding_model.encode(df['answer_llm'][i]))
    score = orig_ans.dot(llm_ans)
    evaluations.append(score)

100%|██████████| 300/300 [02:23<00:00,  2.10it/s]


In [34]:
# Cosine's percentile
percentile_75 = calculate_percentile(cosine_evaluations, 0.75)
print("75th Percentile:", percentile_75)

IndexError: list index out of range

In [52]:
v1 = embedding_model.encode(df['answer_orig'][0])
norma1 = normalize_vector(v1)

In [53]:
v2 = embedding_model.encode(df['answer_orig'][1])
norma2 = normalize_vector(v2)

In [54]:
(max(v1),max(v2))

(0.74044466, 0.74044466)

In [55]:
(max(norma1),max(norma2))

(0.12640719, 0.12640719)

In [56]:
score = norma1.dot(norma2)

In [57]:
score

1.0